In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("MY_OPENAI_API_KEY")
AZURE_DOC_API = os.getenv("AZURE_DOC_API")
AZURE_DOC_ENDPOINT = os.getenv("AZURE_DOC_ENDPOINT")


In [2]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
# text_documents

In [23]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = os.getenv("TEST_FILE")
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=AZURE_DOC_ENDPOINT, api_key=AZURE_DOC_API, file_path=file_path, api_model="prebuilt-layout"
)

documents = loader.load()

In [20]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [21]:
query = "Questions"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

9th Grade Mathematics Test: Volume Calculation
===

Instructions: Answer all the questions. For multiple-choice questions, circle the correct answer. For fill-in-the-blank and show-your-work questions, write your answer clearly in the space provided.


## Question 1 (Multiple Choice):

What is the formula for calculating the volume of a cylinder?

A) π r 2 h

B) π r 2

C) 2πrh

D) 2πr(h + r)


## Question 2 (Fill in the Blank):

The volume of a cube is given by the formula \_\_\_\_\_\_\_\_\_\_, where s represents the length of one side of the cube.


## Question 3 (Show Your Work):

Calculate the volume of a rectangular prism with a length of 10 cm, a width of 4 cm, and a height of 5 cm. Show your work.


## Question 4 (Multiple Choice):

A sphere has a radius of 3 meters. What is the volume of the sphere?

A) 36π m 3

B) 4π m 3

C) 113.1 m 3 (Use 4/3πr 3 for calculation)

D) 27π m 3


## Question 5 (Text Answer):

Explain in your own words why the formula for the volume of a pyramid i

In [4]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()


In [5]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('test.pdf')
docs=loader.load()

In [6]:
docs

[Document(page_content='RepairAgent: An Autonomous, LLM-Based\nAgent for Program Repair\nIslem Bouzenia\nUniversity of Stuttgart\nGermany\nfibouzenia@esi.dzPremkumar Devanbu\nUC Davis\nUSA\nptdevanbu@ucdavis.eduMichael Pradel\nUniversity of Stuttgart\nGermany\nmichael@binaervarianz.de\nAbstract —Automated program repair has emerged as\na powerful technique to mitigate the impact of software\nbugs on system reliability and user experience. This paper\nintroduces RepairAgent, the first work to address the pro-\ngram repair challenge through an autonomous agent based\non a large language model (LLM). Unlike existing deep\nlearning-based approaches, which prompt a model with a\nfixed prompt or in a fixed feedback loop, our work treats\nthe LLM as an agent capable of autonomously planning\nand executing actions to fix bugs by invoking suitable tools.\nRepairAgent freely interleaves gathering information about\nthe bug, gathering repair ingredients, and validating fixes,\nwhile deciding whic

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='RepairAgent: An Autonomous, LLM-Based\nAgent for Program Repair\nIslem Bouzenia\nUniversity of Stuttgart\nGermany\nfibouzenia@esi.dzPremkumar Devanbu\nUC Davis\nUSA\nptdevanbu@ucdavis.eduMichael Pradel\nUniversity of Stuttgart\nGermany\nmichael@binaervarianz.de\nAbstract —Automated program repair has emerged as\na powerful technique to mitigate the impact of software\nbugs on system reliability and user experience. This paper\nintroduces RepairAgent, the first work to address the pro-\ngram repair challenge through an autonomous agent based\non a large language model (LLM). Unlike existing deep\nlearning-based approaches, which prompt a model with a\nfixed prompt or in a fixed feedback loop, our work treats\nthe LLM as an agent capable of autonomously planning\nand executing actions to fix bugs by invoking suitable tools.\nRepairAgent freely interleaves gathering information about\nthe bug, gathering repair ingredients, and validating fixes,\nwhile deciding whic

In [8]:
documents

[Document(page_content='RepairAgent: An Autonomous, LLM-Based\nAgent for Program Repair\nIslem Bouzenia\nUniversity of Stuttgart\nGermany\nfibouzenia@esi.dzPremkumar Devanbu\nUC Davis\nUSA\nptdevanbu@ucdavis.eduMichael Pradel\nUniversity of Stuttgart\nGermany\nmichael@binaervarianz.de\nAbstract —Automated program repair has emerged as\na powerful technique to mitigate the impact of software\nbugs on system reliability and user experience. This paper\nintroduces RepairAgent, the first work to address the pro-\ngram repair challenge through an autonomous agent based\non a large language model (LLM). Unlike existing deep\nlearning-based approaches, which prompt a model with a\nfixed prompt or in a fixed feedback loop, our work treats\nthe LLM as an agent capable of autonomously planning\nand executing actions to fix bugs by invoking suitable tools.\nRepairAgent freely interleaves gathering information about\nthe bug, gathering repair ingredients, and validating fixes,\nwhile deciding whic

In [9]:
## Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [16]:
query = "Better results"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

Fig. 10: Frequency of tool invocations (average per bug).
the Java projects during training. While we acknowl-
edge this risk, our approach does not solely depend on
knowing about a bug, but rather the ability to collect
information to fix the bug. We also note that the closest
competitor, ChatRepair, also uses GPT-3.5, and thus
faces the same risk. (ii) Missing test cases: Defects4J
has at least one failing test case for each bug, which
may not be the case for real-world usage scenarios. It
will be interesting to evaluate RepairAgent on bugs with
no a-priori available error-revealing test cases in future
work. (iii) Fault localization: Inaccurate or imprecise
fault localization could lead to suboptimal repair sug-
gestions or incorrect diagnoses. (iv) Non-deterministic
output of LLMs: The inherently non-deterministic nature
of LLMs may result in different outcomes between two
consecutive runs of RepairAgent. The large number of
bugs we evaluate on mitigates this risk. Moreover, we


In [17]:
## FAISS Vector Database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents[:15], OpenAIEmbeddings())

In [18]:
query = "Better results"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

the LLM and the tools. We present novel techniques for
guiding tool invocations through a finite state machine
and for heuristically interpreting possibly incorrect LLM
outputs. The iterative loop of RepairAgent continues
until the agent declares to have found a suitable fix, or
until exhausting a budget of iterations.
To evaluate the effectiveness of our approach, we
apply it to all 835 bugs in the Defects4J [24] dataset,
a widely used benchmark for evaluating program repair
techniques. RepairAgent successfully fixes 164 bugs,
including 74 and 90 bugs of Defects4J v1.2 and v2.0,
respectively. The correctly fixed bugs include 49 bugs
that require fixing more than one line, showing that
RepairAgent is capable of fixing complex bugs. Com-
pared to state-of-the-art techniques [19], [21], Repair-
Agent successfully fixes 39 bugs not fixed by prior work.
Measuring the costs imposed by interacting with the
LLM, we find that RepairAgent imposes an average cost
